In [6]:
import pandas as pd
import numpy as np

In [7]:
master_filename = 'D:\\Backup\\ISB CBA\\Capstone\\item_master_with_atleast_one_null.csv'
master=pd.read_csv(master_filename)

In [8]:
##de-duplicating features
import difflib
lst=master['case_shape'].unique()
df = pd.Series(data=lst.flatten())
df=df.dropna()
lst=df.to_numpy()
lst1=np.char.lower(lst.tolist())
lst1=np.unique(lst1)
for e in lst:
    lst2=np.delete(lst1,list(lst1).index(e.lower()))
    duplicates=difflib.get_close_matches(e.lower(), lst2)
    if len(duplicates)>1:
        print ([e]+difflib.get_close_matches(e.lower(), lst2))

['Rectangular', 'rectangle', 'rectgle']
['RECTANGLE', 'rectgle', 'rectangular']
['RECTANGULAR', 'rectangle', 'rectgle']
['RECTGLE', 'rectangle', 'rectangular']


In [4]:
# for nulls in item master
master['mrp'] = master['mrp'].str.replace(',', '').astype(float)

lst=[]
cols=['brand','case_size', 'case_size_range', 'gender',
       'movement', 'material', 'dial_color', 'strap_type', 'strap_color',
       'precious_stone', 'glass', 'case_shape', 'watch_type']

for col in cols:      
    agg=master.fillna(0).groupby([col]).agg({'item_no':'count','mrp':'sum','sales_quantity':'sum','null_cnt':'sum'}).sort_values('mrp',ascending=False)
    try:
        count=agg.loc[0][0]
    except:
        continue
    agg=agg/agg.sum()*100
    agg=agg.round(2)
    #print (agg)
    try:
        lst.append([col,count,agg.loc[0][1],agg.loc[0][2],agg.loc[0][3]])
    except:
        continue
        
colm=['Feature','Count', 'MRP%', 'Sales_qty%', 'null_count%']
df = pd.DataFrame(lst, columns=colm)

df=df.sort_values('MRP%', ascending=False)

overall_nulls=master[master.isnull().any(axis=1)]
null_list=['All Nulls',overall_nulls['item_no'].count(),overall_nulls['mrp'].sum(),overall_nulls['sales_quantity'].sum(),overall_nulls['null_cnt'].sum()]
overall_list=['Total',master['item_no'].count(),master['mrp'].sum(),master['sales_quantity'].sum(),master['null_cnt'].sum()]
null_pct= [' All Nulls%',(null_list[1]/overall_list[1]*100).round(2),(null_list[2]/overall_list[2]*100).round(2),(null_list[3]/overall_list[3]*100).round(2),(null_list[4]/overall_list[4]*100).round(2), ]


df.loc[len(df)]=null_list
df.loc[len(df)]=null_pct
df.loc[len(df)]=overall_list
df=df.set_index("Feature")

df.to_excel('D:\\Backup\\ISB CBA\\Capstone\\nulls_summary.xlsx')
df

,Count,MRP%,Sales_qty%,null_count%
Feature,,,,
strap_color,4197.00,7.620000e+00,4.31,94.77
case_size_range,3097.00,6.580000e+00,3.10,88.98
watch_type,1748.00,4.750000e+00,2.19,75.88
glass,1850.00,4.420000e+00,2.67,76.18
dial_color,1352.00,4.300000e+00,1.69,65.83
gender,1418.00,3.400000e+00,1.35,72.89
strap_type,1260.00,3.250000e+00,1.50,65.77
case_shape,1350.00,3.230000e+00,1.06,70.61
material,1177.00,3.040000e+00,1.16,64.85
